# Modeling with StatsModels

## 1. Ordinary Least Square
- Column Names
- Log Transformation
- Condition Number
- Standard Scaling

# 2. Dimensionality Reduction
- ANOVA
- F-test and Feature Influence

# 3. Outlier
- Cook's Distance

# 4. Regularization
- Lasso

# 5. Diagnosis of Regression
- Residual Normality Test
- Partial Regression Plot

# 6. Cross Validatoin

# 7. Test
- score

In [1]:
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import warnings
import sys
import os
import datetime
import scipy as sp
import statsmodels.stats.api as sms
import statsmodels.api as sm
from patsy import dmatrix
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('github'))))
import utils.statsmodel_helper as sh
import utils.feature_selection as fs
import utils.preprocessing as pp
import utils.error_calculator as ec

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

df_train_macro = pd.read_csv('../input/train_macro3.csv', index_col=0)
df_test_macro = pd.read_csv('../input/test_macro3.csv', index_col=0)

sys.setrecursionlimit(1500)

degree = 2
skewness_limit = 1
num_of_cooks = 2
num_of_f_test = 10

# 1. Column Names
## Column Names
Replace -, +, :, ~, * in column name with underscore

In [2]:
new_cols = []
for col in  list(df_train_macro.columns):
    col = col.replace('-', '_').replace('+', '_').replace(':', '_').replace('~', '_').replace('*', '_')
    new_cols.append('_'+col)
df_train_macro.columns = new_cols

new_cols = []
for col in list(df_test_macro.columns):
    col = col.replace('-', '_').replace('+', '_').replace(':', '_').replace('~', '_').replace('*', '_')
    new_cols.append('_'+col)
df_test_macro.columns = new_cols

categorial_ivs = list(set(df_train_macro.columns) - set(df_train_macro._get_numeric_data().columns))
numeric_ivs = df_train_macro._get_numeric_data().columns.drop('_price_doc').tolist()

## Log Transformation
Transform data with skewness greater than 1.

In [3]:
features_to_log = []
for f in df_train_macro._get_numeric_data().columns:
    skewness = sp.stats.skew(df_train_macro[f])
    if skewness > skewness_limit:
        features_to_log.append(f)

for col in df_train_macro._get_numeric_data().columns:
    if col != '_price_doc':
        min_val_train = min(df_train_macro[col])
        min_val_test  = min(df_test_macro[col])
        min_val = min(min_val_train, min_val_test)
        if min_val <= 0:
            df_train_macro[col] += (np.abs(min_val) + 0.1)
            df_test_macro[col]  += (np.abs(min_val) + 0.1)
    else:
        min_val_train = min(df_train_macro[col])
        if min_val_train <= 0:
            df_train_macro[col] += (np.abs(min_val_train) + 0.1)

In [4]:
formula = sh.make_statsmodels_ols_formula(numeric_ivs, categorial_ivs, '_price_doc', log_vs=features_to_log, degree=degree, scale=False)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(_price_doc)   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     580.9
Date:                Tue, 19 Nov 2019   Prob (F-statistic):               0.00
Time:                        01:28:32   Log-Likelihood:                -19234.
No. Observations:               30404   AIC:                         3.855e+04
Df Residuals:                   30365   BIC:                         3.887e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                  0.0030      0.001      4.258      0.000       0.002       0.004
C(_water_1line)[T.yes]                                     0.0078      0.014      0.557      0.578      -0.020       0.035
C(_detention_facility_raion)[T.yes]                        0.0149      0.018      0.814      0.416      -0.021       0.051
C(_railroad_1line)[T.yes]                                 -0.0638      0.020     -3.140      0.002      -0.104      -0.024
C(_thermal_power_plant_raion)[T.yes]                      -0.0208      0.018     -1.125      0.261      -0.057       0.015
C(_oil_chemistry_raion)[T.yes]                            -0.0028      0.009     -0.292      0.771      -0.021       0.016
C(_product_type)[T.OwnerOccupier]                          0.1450      0.013     11.231      0.000       0.120       0.170
C(_radiation_raion)[T.yes]                                -0.0238      0.013     -1.781      0.075      -0.050       0.002
C(_nuclear_reactor_raion)[T.yes]                          -0.0114      0.017     -0.667      0.505      -0.045       0.022
C(_big_market_raion)[T.yes]                                0.0350      0.019      1.866      0.062      -0.002       0.072
C(_big_road1_1line)[T.yes]                                -0.0397      0.019     -2.094      0.036      -0.077      -0.003
C(_ecology)[T.good]                                       -0.0082      0.014     -0.567      0.571      -0.036       0.020
C(_ecology)[T.no data]                                     0.0210      0.009      2.264      0.024       0.003       0.039
C(_ecology)[T.poor]                                        0.0001      0.013      0.009      0.992      -0.025       0.025
C(_ecology)[T.satisfactory]                               -0.0264      0.014     -1.830      0.067      -0.055       0.002
C(_culture_objects_top_25)[T.yes]                         -0.0096      0.020     -0.477      0.634      -0.049       0.030
C(_railroad_terminal_raion)[T.yes]                         0.0394      0.016      2.392      0.017       0.007       0.072
C(_incineration_raion)[T.yes]                             -0.0310      0.015     -2.051      0.040      -0.061      -0.001
np.log(_usdrub)                                            0.0113      0.001     12.794      0.000       0.010       0.013
np.log(_full_sq)                                           0.0067      0.004      1.650      0.099      -0.001       0.015
np.log(_life_sq)                                           0.0133      0.008      1.753      0.080      -0.002       0.028
_floor                                                     0.0076      0.002      4.828      0.000       0.004       0.011
np.log(_

## Condition Number
Large condition number occurs when the scale of data changes significantly due to the unit difference. Scaling can decrease condition number. Multicollinearity can also cause large condition number. We can handle this by reducing dimensionality with variance inflation factor.

## Standard Scaling
Standalize variables by removing the mean and scaling to unit variance.

In [ ]:
formula = sh.make_statsmodels_ols_formula(numeric_ivs, categorial_ivs, '_price_doc', log_vs=features_to_log, degree=degree, scale=True)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(_price_doc)   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     75.25
Date:                Tue, 19 Nov 2019   Prob (F-statistic):               0.00
Time:                        01:30:02   Log-Likelihood:                -19056.
No. Observations:               30404   AIC:                         3.871e+04
Df Residuals:                   30104   BIC:                         4.121e+04
Df Model:                         299                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
Intercept                                                        15.5852      0.027    583.990      0.000      15.533      15.638
C(_water_1line)[T.yes]                                           -0.0006      0.013     -0.047      0.963      -0.026       0.025
C(_detention_facility_raion)[T.yes]                               0.0124      0.021      0.591      0.555      -0.029       0.053
C(_railroad_1line)[T.yes]                                        -0.0694      0.020     -3.449      0.001      -0.109      -0.030
C(_thermal_power_plant_raion)[T.yes]                             -0.0542      0.026     -2.108      0.035      -0.105      -0.004
C(_oil_chemistry_raion)[T.yes]                                   -0.0166      0.050     -0.329      0.742      -0.115       0.082
C(_product_type)[T.OwnerOccupier]                                 0.1518      0.013     11.752      0.000       0.126       0.177
C(_radiation_raion)[T.yes]                                       -0.0446      0.014     -3.302      0.001      -0.071      -0.018
C(_nuclear_reactor_raion)[T.yes]                                  0.0522      0.033      1.564      0.118      -0.013       0.118
C(_big_market_raion)[T.yes]                                       0.0649      0.026      2.492      0.013       0.014       0.116
C(_big_road1_1line)[T.yes]                                       -0.0369      0.020     -1.826      0.068      -0.077       0.003
C(_ecology)[T.good]                                              -0.0377      0.028     -1.336      0.181      -0.093       0.018
C(_ecology)[T.no data]                                            0.0070      0.059      0.119      0.906      -0.109       0.123
C(_ecology)[T.poor]                                              -0.0365      0.020     -1.812      0.070      -0.076       0.003
C(_ecology)[T.satisfactory]                                      -0.0550      0.028     -1.986      0.047      -0.109      -0.001
C(_culture_objects_top_25)[T.yes]                                 0.0178      0.027      0.655      0.512      -0.035       0.071
C(_railroad_terminal_raion)[T.yes]                                0.0737      0.036      2.029      0.042       0.003       0.145
C(_incineration_raion)[T.yes]                                     0.0628      0.039      1.630      0.103      -0.013       0.138
scale(np.log(_usdrub))                                            0.9347      0.107      8.731      0.000       0.725       1.144
scale(np.log(_full_sq))                                           0.0444      0.045      0.996      0.319      -0.043       0.132
scale(np.log(_life_sq))                                           0.0317      0.018      1.78

Scaling did not significantly decrease the condition number.

# 2. Dimensionality Reduction
## ANOVA

In [ ]:
anova = sm.stats.anova_lm(result, typ=2)
anova

,sum_sq,df,F,PR(>F)
C(_water_1line),0.000456,1.0,0.002201,9.625856e-01
C(_detention_facility_raion),0.072293,1.0,0.349033,5.546656e-01
C(_railroad_1line),2.464173,1.0,11.897142,5.629710e-04
C(_thermal_power_plant_raion),0.920428,1.0,4.443868,3.503505e-02
C(_oil_chemistry_raion),0.022443,1.0,0.108356,7.420257e-01
C(_product_type),28.606986,1.0,138.115831,8.069336e-32
C(_radiation_raion),2.257792,1.0,10.900723,9.623857e-04
C(_nuclear_reactor_raion),0.506578,1.0,2.445784,1.178515e-01
C(_big_market_raion),1.286707,1.0,6.212281,1.269204e-02
C(_big_road1_1line),0.690674,1.0,3.334606,6.784656e-02



We can remove features with p-value equal or greater than 0.05 since they have very small influences on the dependent variable

## F-test and Feature Influence

In [ ]:
result, sms_vars, formula = fs.by_f_test(df_train_macro, formula, repeat=num_of_f_test)
result.summary()

# 3. Outlier
## Cook's Distance
- Find data with large leverage and residual by calculating Cook's distance.

In [ ]:
df_train_macro_with_outliers = df_train_macro.copy(deep=True)
df_train_macro, model, result = pp.remove_outliers(df_train_macro, formula, repeat=3)
result.summary()

# 4. Regularization
## Lasso
Find variables with zero coefficient when Lasso regularization is applied.

In [ ]:
result_lasso = model.fit_regularized(alpha=0.001, L1_wt=1)

Let's remove features with zero coefficient to reduce dimensionality.

In [ ]:
sms_vars = []
for idx, coef in enumerate(result_lasso.params):
    if coef ==0:
        continue
    feature = result_lasso.params.index[idx]
    if feature == 'Intercept':
        continue
    startDelPos = feature.find('[')
    endDelPos = feature.find(']')
    feature = feature.replace(feature[startDelPos:endDelPos+1], '')
    sms_vars.append(feature)

In [ ]:
formula = 'np.log(_price_doc) ~ ' + " + ".join(sms_vars)
model = sm.OLS.from_formula(formula, data=df_train_macro)
result = model.fit()
result.summary()

# 5. Diagnosis of Regression
## Residual Normality Test

In [ ]:
# outlier remove result 
sp.stats.probplot(result.resid, plot=plt)
plt.show()

In [ ]:
test = sms.omni_normtest(result.resid)
for xi in zip(['Chi^2', 'P-value'], test):
    print("%-12s: %6.3f" % xi)

# Partial Regression Plot
Let's visualize the influence of a single independent variable.

In [ ]:
fig = plt.figure(figsize=(10,70))
sm.graphics.plot_partregress_grid(result, fig=fig)
fig.suptitle("")
plt.show()

# 6. Cross Validation

In [ ]:
dm = dmatrix(" + ".join(sms_vars) + ' + np.log(_price_doc)', df_train_macro_with_outliers, return_type="dataframe")
X = dm[dm.columns.drop(['np.log(_price_doc)'])]
y = dm['np.log(_price_doc)']
cv = cv = KFold(n_splits=1000, shuffle=True, random_state=0)
r2s = cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2', cv=cv)
r2s.mean()

In [ ]:
plt.hist(r2s, bins=100)

In [ ]:
y_pred = np.exp(result.predict(df_test_macro))
y_pred = y_pred.to_frame('price_doc')
y_pred.to_csv('../submissions/stats_models_{}.csv'.format(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')), header=True, index=True)

## Score

0.40918